What makes DATA so POWERFUL?
It has the power to predict the feature, which is why artificial intelligence is a hot potato

Started with using various APIs from the New York Times, The NEWS, Bloomberg.
However, their data could not go past a few months before. Thus, "The Guardian" was used

Some features to notice:

In [2]:
# def fetch_article
# Note the parameters used for fetch_article
# GET request to retrieve articles
# Function efficiently parses JSON response to extract title, content for the articles

def fetch_articles(api_key, keyword, page_size):
    print(f"Fetching articles for '{keyword}'...")
    base_url = "https://content.guardianapis.com/search"
    params = {
        'api-key': api_key,
        'q': keyword,
        'page-size': page_size,
        'show-fields': 'body',
        'from-date': '2012-01-01',
        'to-date': '2012-12-31'
    }
    constructed_url = requests.Request('GET', base_url, params=params).prepare().url
    print(f"Constructed URL for '{keyword}': {constructed_url}")
   
    try:
        response = requests.get(constructed_url, timeout=10)
        response.raise_for_status()
        data = response.json()
        articles = data['response']['results']
        if not articles:
            print(f"No articles found for '{keyword}'.")
            return []
        print(f"Fetched {len(articles)} articles for '{keyword}'.")
        return [(article['webTitle'], article['fields']['body']) for article in articles if 'body' in article['fields']]
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred while fetching articles for '{keyword}': {http_err}")
    except requests.exceptions.RequestException as err:
        print(f"Request error while fetching articles for '{keyword}': {err}")
    return []

Results for 2020 Election:

![Alt text](images/Screenshot%202024-04-11%20at%203.54.02%20PM.png)

Results for 2016 Election:

![Alt text](images/Screenshot%202024-04-11%20at%203.54.18%20PM.png)

Results for 2012 Election:

![Alt text](images/Screenshot%202024-04-11%20at%203.54.22%20PM.png)